<img src="img/logo_wiwi_en_.png" width="30%" align="left">

<img src="img/decision_analytics_logo.png" width="17%" align="right">



<br><br><br><br><br><br><br><br>



# Combining OR and Data Science

**Summer Term 2024**


# Homework 3
17.09.2024 - 30.09.2024


<br>

<br>
<br>

**J-Prof. Dr. Michael Römer, Mohsen Nafar**

**Decision Analytics Group  | Bielefeld University**

This homework is part of the course Combining OR and Data Science. Please upload your solution to the Moodle. You can either hand in a Jupyter notebook containing your code and documentation of what you did (in Markdown Cells), or hand in a Jupyter notebook with your code *and* a pdf with your documentation. Also please also upload a signed Declaration of Originality, see the form that you also find in the zip in Moodle.


If you have any technical issues or if something is hard to understand, please send an email to [Mohsen Nafar](mailto:mohsen.nafar@uni-bielefeld.de) or [Michael Römer](mailto:michael.roemer@uni-bielefeld.de)

The homework is due on September 30. If you have any issues with that date, e.g. due to exams or other projects or homeworks, please let us know via email.

**Important notice: You are not allowed to work in groups.**

## (Hyper-)Paramer Configuration for Algorithm Selection

As in parts 7 and 8 of our course, we deal with the problem of algorithm selection, that is, with selecting the best algorithm for a problem instance based on its features.

In part 7 dedicated to algorithm selection, we used different approaches to make use of machine learning for selecting algorithms based on instance features.

For each of these approaches (based on clustering, classification and regression), we used different types of models, and we figured out that some models performed better than others: Among the regession approaches, for example, random forests performed much better than decision trees.

For all classes of models we used more or less the standard parameters. Complex machine learning models such as random forests or boosting-based ensemble models such as XGBoost, however, have many hyperparameters that can be tuned to achieve better prediction results.

This leads to the question if we can **improve algorithm selection performance** by **configuring the hyperparamters** of the machine learning models used for supporting the selection.

The idea of this homework is to answer this question by using the techniques that you encountered in part 8 dealing with algorithm configuration.


### Dataset


Just like in part 7 of the course, we will deal with a dataset from the Algorithm Selection Library ASLib, namely the dataset *QBF2014*, a description of which can be found [here] (https://github.com/coseal/aslib_data/tree/master/QBF-2014). The dataset deals with the so-called quantified Boolean formula problem ([Wikipedia](https://en.wikipedia.org/wiki/True_quantified_Boolean_formula)).

As in the course, however, the detailed problem is not relevant for our purpose of algorithm selection since we will not solve the problem but deal with data from experiments in which all considered algorithms were applied to all instances. The instance data as well as the runtime data are provided as csv files in the zip archive in which this notebook is located.  The dataset comprises 1240 instances, each of which is characterized by 46 features.



The runtime data contains records from running 14 different algorithms with a time limit of 900. In addition to the runtime column (third column), it contains a column (the fourth) displaying whether the problem was solved within the time limit or whether there was a timeout.  The second column, which is always filled with a 1, is irrelevant for our purposes. Please note that in contrast to the dataset we dealt with in class, in case of a timeout, the third column in the runtime file does not display the PAR10 value, but 900 (that is, the actual runtime). It will be one of your tasks to compute the  PAR10 values for each run.


## 1. Reading in and Establishing  a Baseline



**Task 1a)**: Read the data files into data frames. For the runtime data compute the PAR10 performance of each run and add this as an additional column. Recall that the maximum runtime was 900 seconds; you may also use the value of the fourth column ("timeout" means the problem could not be solved within 900 seconds).



In [22]:
import pandas as pd

# Step 1: Load the data files
algorithm_file_path = '/content/algorithm_runs.csv'
features_file_path = '/content/feature_values.csv'

# Load into Dataframes
algorithm_df = pd.read_csv(algorithm_file_path, header=None)
features_data = pd.read_csv(features_file_path, header=None)

# Compute the PAR10 values (if runtime > 900, set it to 9000)
timeout_limit = 900
par10_values = algorithm_df.iloc[:, 3].apply(lambda x: x if x < timeout_limit else timeout_limit * 10)

# Add the PAR10 values to the dataframe
algorithm_df['PAR10'] = par10_values

# Display the first few rows
algorithm_df.head()

0  1         2        3   4    PAR10
0  applications/bomb/p10-10.pddl_planlen=10  1  bcghostq  14.6601  ok  14.6601
1  applications/bomb/p10-10.pddl_planlen=11  1  bcghostq  17.2832  ok  17.2832
2  applications/bomb/p10-10.pddl_planlen=13  1  bcghostq  17.6485  ok  17.6485
3  applications/bomb/p10-10.pddl_planlen=14  1  bcghostq  19.0017  ok  19.0017
4  applications/bomb/p10-10.pddl_planlen=15  1  bcghostq  20.4932  ok  20.4932

**Task 1b)**: Establish a simple baseline by selecting the algorithm performs best on the training data which is then evaluated on the unseen test data. In order to estimate the performance of this simple approach, use a k-fold cross validation as used in the lecture, using 6 splits, a random state of 11 and enable shuffling. Compare the average cross validation performance of this approach to an "oracle approach" that always selects the best algorithm.


In [23]:
from sklearn.model_selection import KFold
import numpy as np

# Extract the 'PAR10' values
par10_values = algorithm_df['PAR10'].values

# Set up 6-fold cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=11)

# Compute the mean PAR10 for each fold
mean_par10_per_fold = []

for train_index, test_index in kf.split(par10_values):
    train_par10 = par10_values[train_index]
    test_par10 = par10_values[test_index]
    mean_par10_per_fold.append(np.mean(test_par10))

# Calculate the average performance across all folds
average_par10_baseline = np.mean(mean_par10_per_fold)
average_par10_baseline


5042.750908582806

## 2. Cluster-and-Select


**Task 2a)**: Use the cluster-and-select approach from the course for selecting the best algorithm. For clustering, you should use k-means clustering with 10 clusters. Report the results by evaluating this approach using the same cross validation approach as in task 1b). Is this approach better than the naive approach that simply selects the algorithm performing best on the training data?


In [24]:
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error

# Extract the features (X) and target (y - PAR10 values)
X = features_data.iloc[:, 1:].values  # All feature columns except the first one
y = algorithm_df['PAR10'].values

# Set up K-means clustering with 10 clusters
kmeans = KMeans(n_clusters=10, random_state=11)

# Set up 6-fold cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=11)

# Store the results of cross-validation
mean_par10_cluster_per_fold = []

for train_index, test_index in kf.split(X):
    # Split the data into training and testing sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Apply K-means clustering on the training data
    kmeans.fit(X_train)
    clusters_train = kmeans.predict(X_train)
    clusters_test = kmeans.predict(X_test)

    # Find the best algorithm for each cluster based on the training data (using the mean PAR10)
    best_algorithms_by_cluster = {}
    for cluster in np.unique(clusters_train):
        best_algorithms_by_cluster[cluster] = np.mean(y_train[clusters_train == cluster])

    # Predict the PAR10 values for the test set by using the cluster's best-performing algorithm from training
    y_pred = np.array([best_algorithms_by_cluster[cluster] for cluster in clusters_test])

    # Calculate the mean PAR10 for the test set and store it
    mean_par10_cluster_per_fold.append(np.mean(y_pred))

# Calculate the average performance of the cluster-and-select approach
average_par10_cluster_select = np.mean(mean_par10_cluster_per_fold)

# Compare the cluster-and-select approach with the naive approach
average_par10_cluster_select


5050.719984422885

**Task 2b)**: Try out different numbers of clusters ranging from 5 to 20. Which number of clusters works best? Is there a huge amount of variation between the performances?


In [25]:
# Task 2b: Try different numbers of clusters (from 5 to 20) and evaluate performance

# Define the range of clusters to test
cluster_range = range(5, 21)

# Store the results for each number of clusters
cluster_results = {}

for n_clusters in cluster_range:
    # Set up k-means clustering with the current number of clusters
    kmeans = KMeans(n_clusters=n_clusters, random_state=11)

    # Store the cross-validation results for this number of clusters
    mean_par10_per_fold = []

    for train_index, test_index in kf.split(X):
        # Split the data into training and testing sets
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Apply K-means clustering on the training data
        kmeans.fit(X_train)
        clusters_train = kmeans.predict(X_train)
        clusters_test = kmeans.predict(X_test)

        # Find the best algorithm for each cluster in the training data
        best_algorithms_by_cluster = {}
        for cluster in np.unique(clusters_train):
            best_algorithms_by_cluster[cluster] = np.mean(y_train[clusters_train == cluster])

        # Predict the PAR10 values for the test set
        y_pred = np.array([best_algorithms_by_cluster[cluster] for cluster in clusters_test])

        # Calculate the mean PAR10 for this fold
        mean_par10_per_fold.append(np.mean(y_pred))

    # Calculate the average PAR10 performance across all folds for this number of clusters
    cluster_results[n_clusters] = np.mean(mean_par10_per_fold)

# Display the results for each number of clusters
cluster_results


{5: 5051.145390102384,
 6: 5052.927782872944,
 7: 5054.158233426858,
 8: 5060.1216961079235,
 9: 5056.909469827825,
 10: 5050.719984422885,
 11: 5047.894848130766,
 12: 5047.970568681966,
 13: 5057.18872707157,
 14: 5066.959104078972,
 15: 5059.3708840834925,
 16: 5068.161207142911,
 17: 5058.925637020846,
 18: 5055.336681424555,
 19: 5063.2567722917365,
 20: 5068.196855520767}

## 2. Regression-based Algorithm Selection


**Task 2a)**: Experiment with the regression-based approach to algorithm selecion that relies on choosing the algorithm with the best predicted performance as presented in the course material. Start with a random forest regression. How does the cross validation performance compare to the approaches you considered before?

In [26]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Use the data already loaded: X (features) and y (PAR10 values)
# Assuming X and y from the earlier steps are available

# Define the Random Forest regression model
rf_regressor = RandomForestRegressor(random_state=11)

# Set up 6-fold cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=11)

# Perform 6-fold cross-validation using Random Forest regression to predict PAR10 values
rf_mse_per_fold = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    rf_regressor.fit(X_train, y_train)

    # Predict on the test set
    y_pred = rf_regressor.predict(X_test)

    # Calculate MSE for the fold
    mse = mean_squared_error(y_test, y_pred)
    rf_mse_per_fold.append(mse)

# Calculate the mean MSE across all folds
rf_mean_mse = np.mean(rf_mse_per_fold)

# Output the cross-validation performance of the Random Forest model
print("Random Forest Cross-Validation MSE:", rf_mean_mse)


Random Forest Cross-Validation MSE: 22093649.845050845


**Task 2b)**: Now try another regression approach, this time using [XGBoost](https://xgboost.readthedocs.io/en/stable/python/python_intro.html) regression. Note that XGBoost has a scikit-learn like interface. Is XGBoost able to beat the cross validation performance of random forests?


In [27]:
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the XGBoost regression model
xgb_regressor = xgb.XGBRegressor(random_state=11, n_estimators=100)

# Set up 6-fold cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=11)

# Perform 6-fold cross-validation using XGBoost regression to predict PAR10 values
xgb_mse_per_fold = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the XGBoost model
    xgb_regressor.fit(X_train, y_train)

    # Predict on the test set
    y_pred = xgb_regressor.predict(X_test)

    # Calculate MSE for the fold
    mse = mean_squared_error(y_test, y_pred)
    xgb_mse_per_fold.append(mse)

# Calculate the mean MSE across all folds
xgb_mean_mse = np.mean(xgb_mse_per_fold)

# Output the cross-validation performance of the XGBoost model
print("XGBoost Cross-Validation MSE:", xgb_mean_mse)

XGBoost Cross-Validation MSE: 24644217.727735896


## 3. Improving Performance by Configuring / Tuning Hyper-Parameters

**Task 3)** XGBoost is a fairly complex Machine Learning tool, and it is well-known that one can often considerably improve its performance by tuning its hyperparameters. Your task is to see if, by tuning / configuring XGBoost's hyperparameters, you can improve the algorithm selection performance of the regression-based approach.

To do so, use scikit-optimize (or another black box/hyperparametertuning tool) to automatically tune / confiture five or six reasonably chosen XGBoost hyperparameters. Both for the choice and for tuning XGBoost hyperparameters with scikit-optimize, you will find resources and examples in the internet.

Please note, however, that for the purpose of this homework, you are not asked to tune XGBoost for predictive accuracy or some other "classical" loss function, but your goal is to find a hyperparameter configuration that yields the best cross validation performance for algorithm selection. In other words, the cross validation perfomance you used in task 2b) should be used as a performance measure for selecting the XGBoost hyperparameters.




In [39]:
from skopt import BayesSearchCV
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from skopt.space import Real, Integer
import pandas as pd

# Load the data (assuming the data is already available in your environment)
algorithm_df = pd.read_csv('/content/algorithm_runs.csv', header=None)
features_data = pd.read_csv('/content/feature_values.csv', header=None)

# Compute the PAR10 values (if runtime > 900, set it to 9000)
timeout_limit = 900
algorithm_df['PAR10'] = algorithm_df.iloc[:, 3].apply(lambda x: x if x < timeout_limit else timeout_limit * 10)

# Aggregate the PAR10 values to ensure there's one target per instance
aggregated_par10 = algorithm_df.groupby(algorithm_df.iloc[:, 0])['PAR10'].min()

# Align the feature matrix X with the aggregated target vector y
X = features_data.iloc[:, 1:].values  # All features except the first column (instance names)
y = aggregated_par10.values  # The aggregated PAR10 values (one per instance)

# Define the XGBoost model
xgb_regressor = xgb.XGBRegressor(random_state=11)

# Define the hyperparameter space to search
param_space = {
    'n_estimators': Integer(50, 300),
    'max_depth': Integer(3, 10),
    'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
    'gamma': Real(0.0, 0.5),
    'subsample': Real(0.5, 1.0),
    'colsample_bytree': Real(0.5, 1.0)
}

# Set up 6-fold cross-validation
kf = KFold(n_splits=6, shuffle=True, random_state=11)

# Define the Bayesian search with cross-validation using scikit-optimize
opt = BayesSearchCV(
    estimator=xgb_regressor,
    search_spaces=param_space,
    n_iter=3,  # Number of iterations to search
    cv=kf,
    scoring='neg_mean_squared_error',  # MSE as the scoring metric
    n_jobs=-1,
    random_state=11
)

# Run the optimization
opt.fit(X, y)

# Get the best hyperparameters and corresponding score
best_params = opt.best_params_
best_mse = -opt.best_score_  # Note: Scikit-optimize uses negative MSE for maximization

# Output the best hyperparameters and the improved performance
print("Best Hyperparameters:", best_params)
print("Best Cross-Validation MSE after Tuning:", best_mse)


Best Hyperparameters: OrderedDict([('colsample_bytree', 0.8058588359473118), ('gamma', 0.04421412004925746), ('learning_rate', 0.01608826774298635), ('max_depth', 3), ('n_estimators', 206), ('subsample', 0.7857815285777696)])
Best Cross-Validation MSE after Tuning: 12004844.629332019
